In [ ]:
import torch
import torchvision
from torchvision.models.segmentation import segmentation
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
import time
import matplotlib.image as mpimg
import cv2
import tensorflow as tf
from torchvision import transforms

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/MyDrive/AI_TfLite/

/content/drive/MyDrive/AI_TfLite


In [ ]:
#Set device to GPU_indx if GPU is avaliable
GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')

# Loading the PyTorch Model

In [ ]:
net = segmentation.lraspp_mobilenet_v3_large(pretrained=False, progress=False, num_classes=3).to(device)
print(net)
Pretrained_Path = "./bdd_city_320_mapil_w.pt"
check_point = torch.load(Pretrained_Path)
net.load_state_dict(check_point['model_state_dict'])


In [ ]:
# Requires shape of the image it has been used to train with

x = np.load('val.npz', mmap_mode='r')
image = x['arr_0']
label = x['arr_1']

i=140
ground_image=image[i][:][:][:]
ground_label=label[i][:][:][:]

ground_label=np.squeeze(ground_label)

processed_img_t=np.moveaxis(ground_image,-1,0)
processed_img_t=torch.tensor(processed_img_t).unsqueeze(0)


# Converting PyTorch model to ONNX model

In [ ]:
# Export the model from PyTorch to ONNX
torch_out = torch.onnx._export(net,             # model being run
                                processed_img_t.to(device),          # model input (or a tuple for multiple inputs)
                                "LRASPP.onnx",      # where to save the model (can be a file or file-like object)
                                export_params=True,       # store the trained parameter weights inside the model file
                                input_names=['input'],     # specify the name of input layer in onnx model
                                output_names=['output'])     # specify the name of input layer in onnx model


# Converting ONNX model to Tensorflow model

In [ ]:
import onnx
from onnx_tf.backend import prepare

In [ ]:
!onnx-tf convert -i "LRASPP.onnx" -o  "LRASPP.pb"

## Converting Tensorflow model to TFLite model

In [ ]:
converter =  tf.compat.v1.lite.TFLiteConverter.from_saved_model("LRASPP.pb/")

converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
with tf.io.gfile.GFile('LRASPP.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
x = np.load('val.npz', mmap_mode='r')
image = x['arr_0']
label = x['arr_1']

i=141
ground_image=image[i][:][:][:]
ground_label=label[i][:][:][:]

ground_label=np.squeeze(ground_label)

processed_img_t=np.moveaxis(ground_image,-1,0)
processed_img_t=torch.tensor(processed_img_t).unsqueeze(0)

# Infering TFLite model

In [ ]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="LRASPP.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']


input_data = processed_img_t.numpy()
print(input_data.shape)
# print(input_data.shape)
interpreter.set_tensor(input_details[0]['index'], input_data)
print(input_details)

start_time = time.time()

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
output_data = np.argmax(output_data.squeeze(), axis=0)
end_time = time.time() - start_time


actual_img = input_data.squeeze()
actual_img = actual_img.transpose(1,2,0)
plt.figure(figsize=(15,15))
plt.imshow(actual_img)

image = output_data
plt.figure(figsize=(15,15))
plt.imshow(image)

print(end_time)